In [1]:
using LinearAlgebra, Distributions, Combinatorics, Random, Kronecker, SpecialFunctions
include("../DCM_model/BBVI_utils.jl")

# Object for observed data
struct TDCMObs{T <: AbstractFloat}
    # data
    Y       :: Array{Int, 3}
    Q       :: Matrix{Int}
    D       :: Vector{Matrix{Int}}
    U       :: Vector{Vector{Matrix{T}}}
    X       :: Vector{Vector{Matrix{T}}}
    group   :: Vector{Int}
end

function TDCMObs(
    Y       :: Array{Int, 3}, 
    Q       :: Matrix{Int},
    U       :: Vector{Vector{Matrix{T}}},
    X       :: Vector{Vector{Matrix{T}}},
    group   :: Vector{Int}) where T <: AbstractFloat
    D = generate_delta(Q)
    TDCMObs(Y, Q, D, U, X, group)
end

# Object including latent variables and model parameters
struct TDCMmodel{T <: AbstractFloat}
    # data
    obs         :: TDCMObs
    # prior distribution parameters
    mu_omega_prior      :: Vector{Vector{Vector{Vector{T}}}}
    V_omega_prior       :: Vector{Vector{Vector{Matrix{T}}}}
    a_tau_prior         :: Vector{Vector{Vector{T}}}
    b_tau_prior         :: Vector{Vector{Vector{T}}}
    # This option allocates extra memory based on number of threads availible in the environment
    enable_parallel     :: Bool
    # Variational distribution parameters
    pi_star             :: Vector{Vector{Vector{T}}}
    mu_beta_star        :: Vector{Vector{T}}
    V_beta_star         :: Vector{Matrix{T}}
    mu_gamma_init_star  :: Vector{Vector{Vector{T}}}
    V_gamma_init_star   :: Vector{Vector{Matrix{T}}}
    mu_gamma_t_star     :: Vector{Vector{Vector{Vector{Vector{T}}}}}
    V_gamma_t_star      :: Vector{Vector{Vector{Vector{Matrix{T}}}}}
    mu_omega_star       :: Vector{Vector{Vector{Vector{Vector{T}}}}}
    V_omega_star        :: Vector{Vector{Vector{Vector{Matrix{T}}}}}
    a_tau_star          :: Vector{Vector{Vector{Vector{T}}}}
    b_tau_star          :: Vector{Vector{Vector{Vector{T}}}}
    # Number of samples for noisy gradient
    M                   :: Int
    # Preallocated storage for samples from variational distribution
    Z_sample            :: Vector{Vector{Vector{Vector{Int}}}}
    beta_sample         :: Vector{Vector{Vector{T}}}
    gamma_init_sample   :: Vector{Vector{Vector{Vector{T}}}}
    gamma_sample        :: Vector{Vector{Vector{Vector{Vector{Vector{T}}}}}}
    omega_sample        :: Vector{Vector{Vector{Vector{Vector{Vector{T}}}}}}
    tau_sample          :: Vector{Vector{Vector{Vector{Vector{T}}}}}
    # Preallocated storage for noisy gradient descent calculations
    storage_L           :: Vector{T}
    storage_L2          :: Vector{T}
    storage_L3          :: Vector{T}
    storage_L4          :: Vector{T}
    storage_LL          :: Matrix{T}
    storage_LL2         :: Matrix{T}
    storage_LL3         :: Matrix{T}
    storage_LL4         :: Matrix{T}
    # Preallocated storage for matrix vectorization operations
    storage_comm        :: Matrix{T}
    storage_dup         :: Matrix{T}
    storage_Lsqr        :: Vector{T}
    storage_Lsqr2       :: Vector{T}
    storage_L2L2        :: Matrix{T}
    storage_C           :: Matrix{T}
    storage_gradC       :: Vector{T}
    # Preallocated Identity matrix
    I_LL                :: Matrix{T}
    # Preallocated storage for parallel noisy gradient descent calculations
    storage_L_par       :: Vector{Vector{T}}
    storage_L2_par      :: Vector{Vector{T}}
    storage_L3_par      :: Vector{Vector{T}}
    storage_L4_par      :: Vector{Vector{T}}
    storage_LL_par      :: Vector{Matrix{T}}
    storage_LL2_par     :: Vector{Matrix{T}}
    storage_LL3_par     :: Vector{Matrix{T}}
    storage_LL4_par     :: Vector{Matrix{T}}
    # Preallocated storage for parallel matrix vectorization operations
    storage_comm_par    :: Vector{Matrix{T}}
    storage_dup_par     :: Vector{Matrix{T}}
    storage_Lsqr_par    :: Vector{Vector{T}}
    storage_Lsqr2_par   :: Vector{Vector{T}}
    storage_L2L2_par    :: Vector{Matrix{T}}
    storage_C_par       :: Vector{Matrix{T}}
    storage_gradC_par   :: Vector{Vector{T}}
end

function TDCMmodel(
    obs         :: TDCMObs,
    mu_omega_prior      :: Vector{Vector{Vector{Vector{T}}}},
    V_omega_prior       :: Vector{Vector{Vector{Matrix{T}}}},
    a_tau_prior         :: Vector{Vector{Vector{T}}},
    b_tau_prior         :: Vector{Vector{Vector{T}}};
    # This option allocates extra memory based on number of threads availible in the environment
    enable_parallel     :: Bool=false
) where T <: AbstractFloat
    # Number of students, time points, questions, skills, attribute profiles, groups
    N, O, J, K, L, S = size(obs.Y, 1), size(obs.Y, 2), size(obs.Y, 3),  size(obs.Q, 2), size(obs.D[1], 1), size(obs.U[1][1], 1)
    # Initialize variational distribution parameters
    pi_star = Vector{Vector{Vector{T}}}(undef, N)
    for i in 1:N
        pi_star[i] = Vector{Vector{T}}(undef, O)
        for t in 1:O
            pi_star[i][t] = ones(L)./L
        end
    end
    mu_beta_star = Vector{Vector{T}}(undef, J)
    V_beta_star = Vector{Matrix{T}}(undef, J)
    for i in 1:J
        num_features = size(obs.D[j], 2)
        mu_beta_star[j] = zeros(num_features)
        V_beta_star[j] = Matrix(1.0I, num_features, num_features)
    end
    mu_gamma_init_star = Vector{Vector{Vector{T}}}(undef, K)
    V_gamma_init_star = Vector{Vector{Matrix{T}}}(undef, K)
    for k in 1:K
        mu_gamma_init_star[k] = Vector{Vector{T}}(undef, S)
        V_gamma_init_star[k] = Vector{Matrix{T}}(undef, S)
        for s in 1:S
            num_features = size(obs.X[k][1], 2)
            mu_gamma_init_star[k][s] = zeros(num_features)
            V_gamma_init_star[k][s] = Matrix(1.0I, num_features, num_features)
        end
    end
    mu_gamma_t_star = Vector{Vector{Vector{Vector{Vector{T}}}}}(undef, K)
    V_gamma_t_star = Vector{Vector{Vector{Vector{Matrix{T}}}}}(undef, K)
    for k in 1:K
        mu_gamma_t_star[k] = Vector{Vector{Vector{Vector{T}}}}(undef, O-1)
        V_gamma_t_star[k] = Vector{Vector{Vector{Matrix{T}}}}(undef, O-1)
        for t in 2:O
            mu_gamma_t_star[t][k] = Vector{Vector{Vector{T}}}(undef, )
        end
        for 
    end
end

In [ ]:
function sample_TDCM_variational_distribution(    
    model           :: DCModel;
    sample_Z        :: Bool = false,
    sample_β        :: Bool = false,
    sample_γ        :: Bool = false,
    sample_ω        :: Bool = false,
    sample_τ        :: Bool = true,
    idx_student     :: Int = -1,
    idx_question    :: Int = -1,
    idx_group       :: Int = -1,
    idx_time        :: Int = -1,
    idx_feature     :: Int = -1,
    indicator_skill :: Int = -1
)
M = model.M
if sample_Z
    if idx_time > 0
        if idx_student > 0
            

In [9]:
all([true, true, true])

true

In [2]:
using RCall

R"""
load("TDCM_multilevel_data.RData")
"""
TDCM_data = @rget data
Y = Array{Int, 3}(TDCM_data[:Y])
Q = convert(Matrix{Int64}, TDCM_data[:Q_matrix])
U = Vector{Vector{Matrix{Float64}}}(TDCM_data[:X_group])
for time in TDCM_data[:X_ind]
    for k in 1:length(time)
        if time[k] isa Vector{<: Number}
            time[k] = reshape(time[k], :, 1)
        end
    end
end
X = Vector{Vector{Matrix{Float64}}}(TDCM_data[:X_ind])
group = Vector{Int64}(TDCM_data[:group])
obs = TDCMObs(Y, Q, U, X, group)
;

In [24]:
size(obs.Q, 1)

25

In [28]:
obs.X[1][2]

5000×2 Matrix{Float64}:
 1.0  1.0
 1.0  1.0
 1.0  1.0
 1.0  0.0
 1.0  0.0
 1.0  1.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 ⋮    
 1.0  0.0
 1.0  1.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  1.0
 1.0  1.0
 1.0  1.0
 1.0  1.0
 1.0  1.0
 1.0  1.0
 1.0  1.0